In [56]:
import numpy as np
import os,glob
import scipy.ndimage as sni
import sys
import re
import itertools
from shutil import copytree
from astropy.wcs import WCS
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.visualization import make_lupton_rgb
import aplpy
from astropy.nddata import Cutout2D
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.visualization.wcsaxes import SphericalCircle
from matplotlib.patches import Arrow
from photutils import SkyCircularAnnulus
from photutils import SkyCircularAperture
from photutils import CircularAperture
from photutils import CircularAnnulus
from photutils import SkyEllipticalAperture
from photutils import SkyEllipticalAnnulus
from photutils import EllipticalAperture
from photutils import EllipticalAnnulus
from photutils import aperture_photometry
import numpy.ma as ma
import math
import shutil
import pandas as pd

In [57]:
Dir='/home/heh15/workingspace/Arp240/NGC5258/ratio/'
scriptDir=Dir+'script/measure_1213/'
workDir=Dir+'mask/'
imageDir=Dir+'image/'
logDir=Dir+'log/'
ratioDir=Dir+'1213/contsub/'
maskDir=Dir+'mask/'

In [58]:
# function
def fits_import(fitsimage):
    hdr = fits.open(fitsimage)[0].header
    wcs = WCS(hdr).celestial
    data=fits.open(fitsimage)[0].data
    data=np.squeeze(data)
    data_masked=np.ma.masked_invalid(data)

    return wcs, data_masked

def masked_convert(data_masked,region_masked):
    data_mask=data_masked.mask
    region_mask=np.ma.make_mask(region_masked==0)
    region_mask=np.ma.mask_or(data_mask,region_mask)
    data_region=np.ma.masked_where(region_mask,data_masked)
    return data_region

def flux_mask_get(data_region,rms,chans,chan_width):
    flux=np.ma.sum(data_region)/beam_area_pix
    chans_tmp=chans+np.zeros((np.shape(data_region)[0],np.shape(data_region)[1]))
    error=np.sqrt(chans_tmp)*rms*chan_width/math.sqrt(beam_area_pix)
    error_masked=np.ma.masked_where(data_region.mask,error)
    uncertainty=math.sqrt(np.ma.sum(np.power(error_masked,2)))
    return flux, uncertainty

def ratio_cal(flux1,flux2,uncertainty1,uncertainty2):
    ratio=flux1/flux2
    uncertainty=math.sqrt((uncertainty1/flux1)**2+(uncertainty2/flux2)**2+2*0.05**2)*ratio
    return ratio, uncertainty

In [59]:
beam_area=3.789*2.989*1.1331
beam_area_pix=beam_area/0.09
regions=['south','rest','all']
values=['12CO10 flux','12CO10 uncertainty','13CO10 flux', '13CO10 uncertainty', '12CO21 flux',
        '12CO21 uncertainty','12/13','12/13 uncertainty','21/10','21/10 uncertainty','12CO32 flux','12CO32 uncertainty']
CO13=['13CO10 flux', '13CO10 uncertainty']
ratio_table=pd.DataFrame(index=regions,columns=values)

values2=['12CO10 intensity','12CO10 uncertainty','13CO10 intensity','13CO10 uncertainty',
         '12CO21 intensity','12CO21 uncertainty','12CO32 intensity','12CO32 uncertainty']
intensities=pd.DataFrame(columns=values2,index=regions)
peaks=pd.DataFrame(columns=values2,index=['south'])

## 12CO 1-0

In [60]:
fitsimage=imageDir+'12CO10/NGC5258_12CO10_combine_contsub_uvrange_smooth_co32_pbcor_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

In [61]:
# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0023
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['south']=flux
ratio_table['12CO10 uncertainty']['south']=uncertainty
peaks['12CO10 intensity']['south']=np.nanmax(region)
peaks['12CO10 uncertainty']['south']= peaks['12CO10 intensity']['south']*uncertainty/flux

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0023
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['rest']=flux
ratio_table['12CO10 uncertainty']['rest']=uncertainty

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0023
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['all']=flux
ratio_table['12CO10 uncertainty']['all']=uncertainty

In [62]:
# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['south']=intensity
intensities['12CO10 uncertainty']['south']=intensity*ratio_table['12CO10 uncertainty']['south']\
                                  /ratio_table['12CO10 flux']['south']

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['rest']=intensity
intensities['12CO10 uncertainty']['rest']=intensity*ratio_table['12CO10 uncertainty']['rest']\
                                  /ratio_table['12CO10 flux']['rest']

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['all']=intensity
intensities['12CO10 uncertainty']['all']=intensity*ratio_table['12CO10 uncertainty']['rest']\
                                  /ratio_table['12CO10 flux']['rest']

In [63]:
print(intensities)

      12CO10 intensity 12CO10 uncertainty 13CO10 intensity 13CO10 uncertainty  \
south          9.52107          0.0658402              NaN                NaN   
rest           4.63121          0.0295947              NaN                NaN   
all            5.48699          0.0350634              NaN                NaN   

      12CO21 intensity 12CO21 uncertainty 12CO32 intensity 12CO32 uncertainty  
south              NaN                NaN              NaN                NaN  
rest               NaN                NaN              NaN                NaN  
all                NaN                NaN              NaN                NaN  


## 13CO 1-0 

In [64]:
fitsimage=imageDir+'13CO10/NGC5258_13CO10_12m_uvrange_smooth_co32_pbcor_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

Import the count channel map for 13CO1-0

In [65]:
fitsimage=imageDir+'13CO10/NGC5258_13CO10_12m_uvrange_smooth_co32_nchan.fits'
wcs=fits_import(fitsimage)[0]
chans_masked=fits_import(fitsimage)[1]
chans=chans_masked.data

In [66]:
# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['south']=flux
ratio_table['13CO10 uncertainty']['south']=uncertainty
peaks['13CO10 intensity']['south']=np.nanmax(region)
peaks['13CO10 uncertainty']['south']= peaks['13CO10 intensity']['south']*uncertainty/flux

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
chans=25
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['rest']=flux
ratio_table['13CO10 uncertainty']['rest']=uncertainty

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
chans=25
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['all']=flux
ratio_table['13CO10 uncertainty']['all']=uncertainty

Calculate the intensity of the 13CO 1-0

In [67]:
# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['south']=intensity
intensities['13CO10 uncertainty']['south']=intensity*ratio_table['13CO10 uncertainty']['south']\
                                  /ratio_table['13CO10 flux']['south']

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
chans=25
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['rest']=intensity
intensities['13CO10 uncertainty']['rest']=intensity*ratio_table['13CO10 uncertainty']['rest']\
                                  /ratio_table['13CO10 flux']['rest']

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
chans=25
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['all']=intensity
intensities['13CO10 uncertainty']['all']=intensity*ratio_table['13CO10 uncertainty']['rest']\
                                  /ratio_table['13CO10 flux']['rest']

In [68]:
print(intensities)

      12CO10 intensity 12CO10 uncertainty 13CO10 intensity 13CO10 uncertainty  \
south          9.52107          0.0658402         0.541093          0.0140979   
rest           4.63121          0.0295947         0.286571          0.0140049   
all            5.48699          0.0350634         0.332065          0.0162283   

      12CO21 intensity 12CO21 uncertainty 12CO32 intensity 12CO32 uncertainty  
south              NaN                NaN              NaN                NaN  
rest               NaN                NaN              NaN                NaN  
all                NaN                NaN              NaN                NaN  


## 12CO21

In [69]:
fitsimage=imageDir+'12CO21/NGC5258_12CO21_combine_uvtaper_smooth_co32_pbcor_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

In [70]:
# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0098
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['south']=flux
ratio_table['12CO21 uncertainty']['south']=uncertainty
peaks['12CO21 intensity']['south']=np.nanmax(region)
peaks['12CO21 uncertainty']['south']= peaks['12CO21 intensity']['south']*uncertainty/flux

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0098
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['rest']=flux
ratio_table['12CO21 uncertainty']['rest']=uncertainty

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0098
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['all']=flux
ratio_table['12CO21 uncertainty']['all']=uncertainty

In [71]:
# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['south']=intensity
intensities['12CO21 uncertainty']['south']=intensity*ratio_table['12CO21 uncertainty']['south']\
                                  /ratio_table['12CO21 flux']['south']

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['rest']=intensity
intensities['12CO21 uncertainty']['rest']=intensity*ratio_table['12CO21 uncertainty']['rest']\
                                  /ratio_table['12CO21 flux']['rest']

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['all']=intensity
intensities['12CO21 uncertainty']['all']=intensity*ratio_table['12CO21 uncertainty']['rest']\
                                  /ratio_table['12CO21 flux']['rest']

In [72]:
ratio_table['12/13']=ratio_table['12CO10 flux']/ratio_table['13CO10 flux']
fraction=((ratio_table['12CO10 uncertainty']/ratio_table['12CO10 flux'])**2+
                                  (ratio_table['13CO10 uncertainty']/ratio_table['13CO10 flux'])**2)**0.5
ratio_table['12/13 uncertainty']=ratio_table['12/13']*fraction

In [73]:
ratio_table['21/10']=ratio_table['12CO21 flux']/ratio_table['12CO10 flux']*115.27**2/230.54**2
fraction=((ratio_table['12CO21 uncertainty']/ratio_table['12CO21 flux'])**2+
                                  (ratio_table['12CO10 uncertainty']/ratio_table['12CO10 flux'])**2)**0.5
ratio_table['21/10 uncertainty']=ratio_table['21/10']*fraction
print(ratio_table)

      12CO10 flux 12CO10 uncertainty 13CO10 flux 13CO10 uncertainty  \
south     58.0937            0.40173     3.14594          0.0819655   
rest       139.86           0.893741     7.78601           0.380509   
all       197.605           0.975989     10.9294           0.418803   

      12CO21 flux 12CO21 uncertainty    12/13 12/13 uncertainty     21/10  \
south     210.209            1.71172  18.4662          0.497784  0.904612   
rest      453.564            3.80811  17.9629          0.885335  0.810749   
all       662.408            4.15856  18.0801          0.698539  0.838044   

      21/10 uncertainty 12CO32 flux 12CO32 uncertainty  
south        0.00966402         NaN                NaN  
rest         0.00855438         NaN                NaN  
all          0.00669425         NaN                NaN  


In [74]:
print(intensities)

      12CO10 intensity 12CO10 uncertainty 13CO10 intensity 13CO10 uncertainty  \
south          9.52107          0.0658402         0.541093          0.0140979   
rest           4.63121          0.0295947         0.286571          0.0140049   
all            5.48699          0.0350634         0.332065          0.0162283   

      12CO21 intensity 12CO21 uncertainty 12CO32 intensity 12CO32 uncertainty  
south          34.4515           0.280537              NaN                NaN  
rest            15.019           0.126099              NaN                NaN  
all            18.3934            0.15443              NaN                NaN  


## 12CO32

The wcs of regridded image can't be imported somehow. So I just import the data and use the wcs from previous imported image.

In [75]:
def fits_import(fitsimage):
    
    data=fits.open(fitsimage)[0].data
    data=np.squeeze(data)
    data_masked=np.ma.masked_invalid(data)
    
    return data_masked

In [76]:
fitsimage=imageDir+'12CO32/NGC5258co32_all_regrid.map40r.mom0.fits'
data_masked=fits_import(fitsimage)

In [77]:
np.max(data_masked)

161.2186

In [78]:
# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)
region= masked_convert(data_masked,region_masked)
rms=0.165
chan_width=40
chans=7
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO32 flux']['south']=flux
ratio_table['12CO32 uncertainty']['south']=uncertainty

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)
region= masked_convert(data_masked,region_masked)
rms=0.165
chan_width=40
chans=7
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO32 flux']['rest']=flux
ratio_table['12CO32 uncertainty']['rest']=uncertainty

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)
region= masked_convert(data_masked,region_masked)
rms=0.165
chan_width=40
chans=7
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO32 flux']['all']=flux
ratio_table['12CO32 uncertainty']['all']=uncertainty

In [79]:
print(ratio_table)

      12CO10 flux 12CO10 uncertainty 13CO10 flux 13CO10 uncertainty  \
south     58.0937            0.40173     3.14594          0.0819655   
rest       139.86           0.893741     7.78601           0.380509   
all       197.605           0.975989     10.9294           0.418803   

      12CO21 flux 12CO21 uncertainty    12/13 12/13 uncertainty     21/10  \
south     210.209            1.71172  18.4662          0.497784  0.904612   
rest      453.564            3.80811  17.9629          0.885335  0.810749   
all       662.408            4.15856  18.0801          0.698539  0.838044   

      21/10 uncertainty 12CO32 flux 12CO32 uncertainty  
south        0.00966402     416.078            43.1335  
rest         0.00855438     15.8455            95.9604  
all          0.00669425     431.618            104.791  


In [80]:
# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO32 intensity']['south']=intensity
intensities['12CO32 uncertainty']['south']=intensity*ratio_table['12CO32 uncertainty']['south']\
                                  /ratio_table['12CO32 flux']['south']

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO32 intensity']['rest']=intensity
intensities['12CO32 uncertainty']['rest']=intensity*ratio_table['12CO32 uncertainty']['rest']\
                                  /ratio_table['12CO32 flux']['rest']

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO32 intensity']['all']=intensity
intensities['12CO32 uncertainty']['all']=intensity*ratio_table['12CO32 uncertainty']['rest']\
                                  /ratio_table['12CO32 flux']['rest']

In [81]:
print(intensities)

      12CO10 intensity 12CO10 uncertainty 13CO10 intensity 13CO10 uncertainty  \
south          9.52107          0.0658402         0.541093          0.0140979   
rest           4.63121          0.0295947         0.286571          0.0140049   
all            5.48699          0.0350634         0.332065          0.0162283   

      12CO21 intensity 12CO21 uncertainty 12CO32 intensity 12CO32 uncertainty  
south          34.4515           0.280537          68.1917            7.06922  
rest            15.019           0.126099         0.524698            3.17756  
all            18.3934            0.15443          11.9849            72.5805  
